## Regularization Methods Example 1.2:
In the following, we will discuss the function **sklearn.linear\_model.Ridge()** in depth. Firstly, note that the qualitative predictors in x have to be transformed into dummy variables. The flag **normalize = True** makes sure that the predictors are mean centred and scaled to unit variance. When comparing to **R**, note that the implementation is slightly different, which makes it hard to compare coefficients as a function of lambda. The optimal solution however, will generally be the same. 

We will now perform ridge regression in order to predict **Balance** in the **Credit** data set.

In [1]:
import pandas as pd
import numpy as np
from sklearn.linear_model import Ridge
import warnings
warnings.filterwarnings("ignore")

# Load data
df = pd.read_csv('./data/Credit.csv', index_col="Unnamed: 0")

# Convert Categorical variables
df = pd.get_dummies(data=df, drop_first=True, 
                    prefix=('Gender_', 'Student_', 
                            'Married_', 'Ethnicity_'))

# Define target and predictors
x = df.drop(columns='Balance') 
y = df['Balance']

# Fit model:
lambda_ = 100
reg = Ridge(alpha=lambda_) #, normalize=True) normalize was removed in recent versions of scikit-learn
reg = reg.fit(x, y)

# Coefficient and corresponding predictors
coef = np.round(reg.coef_, 3)
# coef = scaler.inverse_transform(coef)
x_cols = x.columns.values

We expect the coefficient estimates to be much smaller, in terms of $\ell_2$ norm, when a large value of $\lambda$ is used, as compared to when a small value of $\lambda$ is used. These are the coefficients when $\lambda = 100$, along with their $\ell_2$ norm:


In [2]:
print(pd.DataFrame(data={'Feature': x_cols,
                         'Coefficient':coef}),
      '\n\nl2-norm:', np.sqrt(np.sum(coef**2)))

                 Feature  Coefficient
0                 Income       -7.624
1                  Limit        0.131
2                 Rating        2.007
3                  Cards       10.820
4                    Age       -0.831
5              Education        1.210
6           Gender__Male        0.053
7           Student__Yes      110.837
8           Married__Yes      -12.679
9       Ethnicity__Asian        9.062
10  Ethnicity__Caucasian       -1.347 

l2-norm: 112.74277058419312


In contrast, here are the coefficients when $\lambda = 50$, along with their $\ell_2$ norm. Note the much larger $\ell_2$ norm of the coefficients associated with this smaller value of $\lambda$.

In [4]:
# Fit model:
lambda_ = 50
reg = Ridge(alpha=lambda_) #, normalize=True)
reg = reg.fit(x, y)

# Coefficient and corresponding predictors
coef = np.round(reg.coef_, 3)
x_cols = x.columns.values

print(pd.DataFrame(data={'Feature': x_cols,
                         'Coefficient':coef}),
      '\n\nl2-norm:', np.sqrt(np.sum(coef**2)))

                 Feature  Coefficient
0                 Income       -7.659
1                  Limit        0.143
2                 Rating        1.821
3                  Cards       12.583
4                    Age       -0.793
5              Education        0.769
6           Gender__Male        1.502
7           Student__Yes      175.642
8           Married__Yes      -14.776
9       Ethnicity__Asian       12.068
10  Ethnicity__Caucasian        0.478 

l2-norm: 177.30795899225728


The standard least squares coefficient estimates are scale equivariant: multiplying a predictor variable $X_j$ by a constant $c$ simply leads to a scaling of the least squares coefficient estimates by a factor of $1/c$. In other words, regardless of how the $j$th predictor is scaled, $\hat{\beta}_j X_j $ will remain the
same. In contrast, the ridge regression coefficient estimates can change substantially when multiplying a given predictor by a constant. For instance, consider the **income** variable, which is measured in dollars. One could reasonably have measured income in thousands of dollars, which would result in a reduction in the observed values of income by a factor of $1000$. Now due to the sum of squared coefficients in the ridge regression formulation equation, such a change in scale will not simply cause the ridge regression coefficient estimate for **income** to change by a factor of $1000$. In other words,
$\hat{\beta}_j X_{j,\lambda}^{R} $ will depend not only on the value of $\lambda$, but also on the scaling of the $j$th predictor. In fact, the value of $\hat{\beta}_j X_{j,\lambda}^{R} $ may even depend on the scaling of the other predictors. Therefore, it is best to apply ridge regression after standardizing the predictors.

Note that by default, the **Ridge()** function does not standardizes the variables. To turn on scaling, use the argument **normalize = True**.